We implement from scratch a indexed heap data structure used in DijkstraShortestPaths algorithm since it needs to modify the priority of a certain node 

Passed all test cases at this hackerrank challenge https://www.hackerrank.com/challenges/dijkstrashortreach/problem?h_r=internal-search  
The key is all edge weights are non-negative

In [58]:
import time


In [39]:
# Create graph, only keeping the shortest edge if there are multiple e:v->w 
# Since the hackerrank problem is undirected 
def create_graph(n, edges):
    adj_list = [{} for _ in range(n)]
    for s,e,w in edges:
        # input data is base-1 index
        s = s - 1
        e = e - 1
        if e in adj_list[s]:
            adj_list[s][e] = min([adj_list[s][e], w])
            adj_list[e][s] = min([adj_list[e][s], w])
        else:
            adj_list[s][e] = w
            adj_list[e][s] = w
    return adj_list

In [40]:
# Naive version without using heap
def shortestReach_naive(n, edges, s):
    # s is starting from 1
    s = s -1
    graph = create_graph(n, edges)
    distTo = [-1 for _ in range(n)] # -1 denotes unreachable 
    
    array = set()
    
    # start from the source
    distTo[s] = 0.0 
    array.add(s)    
    while len(array)>0:
        # figure out the smallest number
        min_dist = None
        argmin_dist = None
        for k in array:
            if min_dist is None or min_dist > distTo[k]:
                min_dist = distTo[k]
                argmin_dist = k
        
        v = argmin_dist
        
        array.discard(v)
        
        for e in graph[v]:
            # relax edge e
            if distTo[e] == -1  or distTo[e] > distTo[v] + graph[v][e]:
                # previsouly discarded v will never become e here since those v are finalized solution and due to positive edge, we cannot find an improvement                
                distTo[e] = distTo[v] + graph[v][e]
                if e not in array:
                    array.add(e)
    
    # print out solution by order ,ignoring s
    out = distTo[:s] + distTo[(s+1):]
    
    # convert out to integer
    out = [int(temp) for temp in out]
    
    return(out)
                
        
            
    

In [59]:
fptr = open("./data/DijkstraShortestPaths_testcase7", 'r')

t = int(fptr.readline().rstrip())

for t_itr in range(t):
    nm = fptr.readline().rstrip().split(" ")

    n = int(nm[0])

    m = int(nm[1])

    edges = []

    for i in range(m):

        edges.append(list(map(int, fptr.readline().rstrip().split(" "))))

    s = int(fptr.readline().rstrip())
    
    # time the shortest path runtime 
    start_time = time.time()
    result = shortestReach_naive(n, edges, s)
    shortestPathsRuntime = time.time()- start_time
fptr.close()
print(shortestPathsRuntime)

2.69574236869812


Testcase 7 from Hackerrank

In [ ]:
with open("./data/DijkstraShortestPaths_testcase7_sol", 'r') as f:
    solution = list(map(int, f.readline().rstrip().split(" ")))

In [51]:
len(solution) == len(result)

True

In [55]:
for i in range(len(solution)):
    wrong = 0 
    if solution[i]!= result[i]:
        print(str(i) + "th solution is wrong!")
        print(solution[i])
        print(result[i])
        wrong += 1
if wrong ==0:
    print("correct!")


correct!


We next try to improve the runtime from O(V^2) to O(Elog(V)) using indexed priority queue implemented via binary heap

In [71]:
class IndexPQ():
    # MaxPQ
    def __init__(self, maxN):
        # maxN is the maximum number of objects we are dealing with. 
        # We represent the maxN objects as 0, 1, ..., maxN - 1. We call this object index which starts from 0 
        self.pq = [None for _ in range(maxN + 1)] # the priority queue using 1-base index
        self.pq[0] = 0 # 0th position is used for actual size of pq used right now 
        self.qp = [None for _ in range(maxN)] # from object index to pq positions 
        self.Keys = [None for _ in range(maxN)] # from object index to its key, which decides priority 
    
    def size(self):
        return self.pq[0]
    
    # less 
    def less(self, pos_idx1, pos_idx2):
        return self.Keys[self.pq[pos_idx1]] < self.Keys[self.pq[pos_idx2]]
    
    
    
    def exch(self, pos_idx1, pos_idx2):
        temp =self.pq[pos_idx1]
        self.pq[pos_idx1] = self.pq[pos_idx2]
        self.pq[pos_idx2] = temp
        
        self.qp[self.pq[pos_idx1]] = pos_idx1
        self.qp[self.pq[pos_idx2]] = pos_idx2
    
    # contain
    def contain(self, obj_idx):
        return(self.qp[obj_idx] is not None)
        
    # sink operation 
    def sink(self, pos_idx):
        
        
        while pos_idx * 2 <= self.pq[0]:
            
            
            pq_child_pos = 2*pos_idx
            if self.pq[0] >= pq_child_pos + 1 and self.less(pq_child_pos, pq_child_pos + 1):
                pq_child_pos = pq_child_pos + 1
            # at this point pq_child_pos is the larger child of the two children nodes of pq_pos
            if ~self.less(pos_idx, pq_child_pos):
                # we dont have to sink anymore
                break
            self.exch(pos_idx, pq_child_pos)
            
            # we dont have change obj_idx anymore, since that is still the index of the object we want to sink
            
     # swim operation 
    def swim(self, pos_idx):                
        while int(pos_idx/2) > 1 and self.less(int(pos_idx/2), pos_idx) :
            self.exch(int(pos_idx/2), pos_idx)
            pos_idx = int(pos_idx/2) 
            
    
    def insert(self, idx_a, key_a):
        # insert key_a
        if self.contain(idx_a):
            print("PQ has this object already, error!")
            return        
        self.pq[0] += 1
        self.pq[self.pq[0]] = idx_a
        self.qp[idx_a] = self.pq[0]
        self.Keys[idx_a] = key_a
        
        # maintain heap property
        self.swim(self.pq[0])
    
    def queryMaxIndex(self):
        return self.pq[1]
    
    def queryMaxKey(self):
        return self.Keys[self.queryMaxIndex()]
    
    
    def delMax(self):
        # swap max and the last item, decrease size
        maxIndex = self.queryMaxIndex()
        maxKey = self.queryMaxKey()
        self.exch(1, self.pq[0])
        # delete the last item from the queue
        self.pq[self.pq[0]] = None
        self.qp[maxIndex] = None
        self.Keys[maxIndex] = None        
        self.pq[0] -= 1
        
        # Regain heap property 
        self.sink(1)
        
        return maxIndex, maxKey
    
    def changePriority(self, index_a, new_key_a):
        if self.Keys[index_a] > new_key_a:
            # decrease key
            self.Keys[index_a] = new_key_a
            self.sink(self.qp[index_a])
        if self.Keys[index_a] < new_key_a:
            # increase key
            self.Keys[index_a] = new_key_a
            self.swim(self.qp[index_a])
        

In [72]:
def shortestReach_IndexPQ(n, edges, s):
    # s is starting from 1
    s = s -1
    graph = create_graph(n, edges)
    distTo = [-1 for _ in range(n)] # -1 denotes unreachable     
    maxPQ = IndexPQ(n)
    
    # start from the source
    distTo[s] = 0.0 
    maxPQ.insert(s, 0.0)    
    while maxPQ.size()>0:
        # figure out next point on the solution tree
        argmin_dist, min_dist = maxPQ.delMax()
        min_dist = - min_dist 
        # The discarded node is added on the final solution tree and is finalized
        # this min_dist should equal to the current corresponding item on distTo and this will never be modified again as long as edge weights are >=0
        
        v = argmin_dist
        
        for e in graph[v]:
            # relax edge e
            if distTo[e] == -1  or distTo[e] > distTo[v] + graph[v][e]:
                # previsouly discarded v will never become e here since those v are finalized solution and due to positive edge, we cannot find an improvement                
                distTo[e] = distTo[v] + graph[v][e]
                if maxPQ.contain(e):
                    maxPQ.changePriority(e, -distTo[e])
                else:
                    maxPQ.insert(e, -distTo[e])
    
    # print out solution by order ,ignoring s
    out = distTo[:s] + distTo[(s+1):]
    
    # convert out to integer
    out = [int(temp) for temp in out]
    
    return(out)
                

In [77]:
fptr = open("./data/DijkstraShortestPaths_testcase7", 'r')

t = int(fptr.readline().rstrip())

for t_itr in range(t):
    nm = fptr.readline().rstrip().split(" ")

    n = int(nm[0])

    m = int(nm[1])

    edges = []

    for i in range(m):

        edges.append(list(map(int, fptr.readline().rstrip().split(" "))))

    s = int(fptr.readline().rstrip())
    
    # time the shortest path runtime 
    start_time = time.time()
    result_naive = shortestReach_naive(n, edges, s)
    shortestPathsRuntime_naive = time.time()- start_time
    
    start_time = time.time()
    result_IndexPQ = shortestReach_IndexPQ(n, edges, s)
    shortestPathsRuntime_IndexPQ = time.time()- start_time
fptr.close()
print(shortestPathsRuntime_naive)
print(shortestPathsRuntime_IndexPQ)

2.646946668624878
2.5252461433410645


The difference in time is actually not that big. As a result the hackerrank test still times out. But at least for the only timeout case, we could check that both our implementation gives the correct results. 

Check whether our indexPQ solutino is correct or not

In [76]:
for i in range(len(solution)):
    wrong = 0 
    if solution[i]!= result_IndexPQ[i]:
        print(str(i) + "th solution is wrong!")
        print(solution[i])
        print(result_IndexPQ[i])
        wrong += 1
if wrong ==0:
    print("correct!")

correct!
